__Import dependencies and decleare global variables__

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd
import json
import re
import sys
import os
import time

SEQUENCES_LENGTH = 30
EPOCHS = 3
BATCH_SIZE = 32
EMBEDDING_DIM = 128
RNN_DIM = 1024 
NUM_PLOTS = 5000

__Import fables data__

In [ ]:
def clean(text):
    '''
    '''
    text = text.strip()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace(",", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("-", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])

    return text

try:
    
    dirname = os.path.abspath('')
    filepath = os.path.join(dirname, 'input_data/wiki_movie_plots.csv')
    dataframe = pd.read_csv(filepath, sep=',')
    plotList = dataframe['Plot']
    plotList = plotList[:NUM_PLOTS]
    plotText = ''
    print('{} plots imported.'.format(len(plotList)))
    for idx , p in enumerate(plotList):
        cleaned = clean(p)
        plotText += ' ' + cleaned + '\n'
    
    print('{} plots cleaned.'.format(len(plotList)))
    
except IOError:
    sys.exit('Cannot find data!')


__Extract Vocabulary__

In [ ]:
# CREATE VOCABULARY OF WORDS
idx2word = []
word2idx = {'<PAD>' : 0, '<START>' : 1 , '<END>': 2}
wordSequence = []
for plot in plotList:
    words = plot.split(' ')
    wordSequence.extend(words)
    for word in words:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

for word in idx2word:
    word2idx[word] = len(word2idx)

idx2word = list(word2idx.keys())
textAsInt = np.array([word2idx[w] for w in wordSequence])
vocab_size = len(idx2word)
print('Vocabulary Size: {}'.format(vocab_size))


__Preprocess__

In [ ]:
def split_input_target(chunk):
    '''
    '''
    inputText = chunk[:-1]
    targetText = chunk[1:]
    return inputText, targetText

wordDataset = tf.data.Dataset.from_tensor_slices(textAsInt)
sequences = wordDataset.batch(SEQUENCES_LENGTH+1, drop_remainder=True) #The batch method lets us easily convert these individual characters to sequences of the desired size.
dataset = sequences.map(split_input_target)

examplesPerEpoch = len(wordSequence) // SEQUENCES_LENGTH
stepsPerEpoch = examplesPerEpoch // BATCH_SIZE
dataset = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)
dataset

__Extract embeddings matrix__

In [11]:
# Recreating embeddings index based on Tokenizer vocabulary
word2vecModel = gensim.models.Word2Vec.load('embeddings/text8_word2vec_skipgram_128.bin')
word2vec_vocabulary = word2vecModel.wv.vocab
embeddingIndex = dict()
counter = 0
for i, word in enumerate(idx2word):
    if word in word2vec_vocabulary :
        embeddingIndex[word] = word2vecModel[word]
    else:
        counter += 1

print("{} words without pre-trained embedding!".format(counter))
    
# Prepare embeddings matrix
embeddingMatrix = np.random.random((len(word2idx), EMBEDDING_DIM))
for i, word in enumerate(idx2word):
    embeddingVector = embeddingIndex.get(word)
    if embeddingVector is not None:
        embeddingMatrix[i] = embeddingVector

107 words without pre-trained embedding!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


__Or use random weights__

In [ ]:
embeddingMatrix = np.random.random((len(word2idx), EMBEDDING_DIM))

__Train the model__

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [ ]:
def loss(labels, logits):
  '''
  '''
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits)

rnn = tf.keras.layers.CuDNNLSTM

trainModel = tf.keras.Sequential(
    [tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM, weigths=[embeddingMatrix], batch_input_shape=[BATCH_SIZE, None]),
    rnn(RNN_DIM,return_sequences=True, recurrent_initializer='glorot_uniform',stateful=True),
    tf.keras.layers.Dense(vocab_size)]
)

trainModel.summary()

trainModel.compile(
      optimizer = tf.train.AdamOptimizer(),
      loss = loss)

trainModel.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=stepsPerEpoch)

dirname = os.path.abspath('')
weightsPath = os.path.join(dirname, 'models/rnn_word_fables.h5')
trainModel.save_weights(weightsPath)

__Define generation model__

In [ ]:
rnn = tf.keras.layers.CuDNNLSTM

genModel = tf.keras.Sequential(
    [tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM, 
    batch_input_shape=[1, None]),
    rnn(RNN_DIM,return_sequences=True, recurrent_initializer='glorot_uniform',stateful=True),
    tf.keras.layers.Dense(vocab_size)]
)

dirname = os.path.abspath('')
weightsPath = os.path.join(dirname, 'models/rnn_word_fables.h5')
genModel.load_weights(weightsPath)
genModel.build(tf.TensorShape([1, None]))
genModel.summary()

__Generate text__

In [ ]:
def generate_text(model, start_string, word_2_idx, idx_2_word):
    '''
    '''
    # Evaluation step (generating text using the learned weights)
    # Number of characters to generate
    numGenerate = 30
    # Converting our start string to numbers (vectorizing) 
    s = clean(start_string)
    inputEval = [word_2_idx[w] for w in s.split(' ')]
    inputEval = tf.expand_dims(inputEval, 0)
    # Empty string to store our results
    textGenerated = []
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    # Here batch size == 1
    model.reset_states()

    for i in range(numGenerate):
        predictions = model(inputEval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a multinomial distribution to predict the word returned by the trainModel
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].eval(session = session)
        # We pass the predicted word as the next input to the trainModel
        # along with the previous hidden state
        inputEval = tf.expand_dims([predicted_id], 0)
        textGenerated.append(idx_2_word[predicted_id])

    return (start_string + ' '.join(textGenerated))

generated = generate_text(
        model=genModel, 
        start_string="There was once a little", 
        word_2_idx=word2idx, 
        idx_2_word=idx2word
    )

generated

In [ ]:
session.close()